In [1]:
import polars as pl

In [2]:
df = pl.read_parquet('full_results_with_date2.parquet')

In [3]:
df = df.with_columns([
    (pl.col("Combined") - pl.col("Ground Truth")).abs().alias("Abs Diff")
])

In [4]:
len(df)

213003

In [5]:
df.head()

Beach,Image,Date,Seal Conf Lvl,Clump Conf Lvl,Overlap,Number of Clumps,Number of Seals,Number of Nonintersec Seals,Number of Seals from Clumps,Combined,Ground Truth,Abs Diff
str,str,datetime[ns],i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""LN""","""DJI_0781""",2023-02-13 06:48:24,10,10,10,0,95,95,0,95,50,45
"""LN""","""DJI_0781""",2023-02-13 06:48:24,14,10,10,0,85,85,0,85,50,35
"""LN""","""DJI_0781""",2023-02-13 06:48:24,18,10,10,0,75,75,0,75,50,25
"""LN""","""DJI_0781""",2023-02-13 06:48:24,22,10,10,0,69,69,0,69,50,19
"""LN""","""DJI_0781""",2023-02-13 06:48:24,26,10,10,0,63,63,0,63,50,13


In [6]:
df.filter(pl.col('Number of Clumps') != 0)

Beach,Image,Date,Seal Conf Lvl,Clump Conf Lvl,Overlap,Number of Clumps,Number of Seals,Number of Nonintersec Seals,Number of Seals from Clumps,Combined,Ground Truth,Abs Diff
str,str,datetime[ns],i64,i64,i64,i64,i64,i64,i64,i64,i64,i64


In [10]:
df.sort("Abs Diff").group_by("Image").agg([
          pl.first("Seal Conf Lvl").alias("Best Seal Conf Lvl"),
          pl.first("Clump Conf Lvl").alias("Best Clump Conf Lvl"),
          pl.first("Abs Diff").alias("Smallest Abs Diff")
      ])

Image,Best Seal Conf Lvl,Best Clump Conf Lvl,Smallest Abs Diff
str,i64,i64,i64
"""DJI_0925""",10,10,11
"""DJI_0223""",38,10,0
"""DJI_0977""",10,10,50
"""DJI_0786""",38,10,0
"""DJI_0781""",42,10,0
…,…,…,…
"""DJI_0789""",30,10,0
"""DJI_0961""",10,10,42
"""DJI_0944""",10,10,0


In [11]:
df.group_by(["Beach", "Seal Conf Lvl", "Clump Conf Lvl", "Overlap"]).agg([
          pl.mean("Abs Diff").alias("Mean Abs Diff")
      ]).sort("Mean Abs Diff").group_by("Beach").agg([
               pl.first("Seal Conf Lvl").alias("Best Seal Conf Lvl"),
               pl.first("Clump Conf Lvl").alias("Best Clump Conf Lvl"),
               pl.first("Overlap").alias("Best Overlap"),
               pl.first("Mean Abs Diff").alias("Smallest Mean Abs Diff")
           ])

Beach,Best Seal Conf Lvl,Best Clump Conf Lvl,Best Overlap,Smallest Mean Abs Diff
str,i64,i64,i64,f64
"""LS""",10,78,90,43.846154
"""DC""",14,58,42,5.2
"""LN""",42,38,34,2.2


In [13]:
df.filter((pl.col('Beach') == 'LN') & (pl.col('Seal Conf Lvl') == 42) & (pl.col('Clump Conf Lvl') == 38) & (pl.col('Overlap') == 34))

Beach,Image,Date,Seal Conf Lvl,Clump Conf Lvl,Overlap,Number of Clumps,Number of Seals,Number of Seals from Clumps,Combined,Ground Truth,Abs Diff
str,str,datetime[ns],i64,i64,i64,i64,i64,i64,i64,i64,i64
"""LN""","""DJI_0781""",2023-02-13 06:48:24,42,38,34,0,50,0,50,50,0
"""LN""","""DJI_0783""",2023-02-13 06:48:36,42,38,34,0,12,0,12,9,3
"""LN""","""DJI_0786""",2023-02-13 06:49:08,42,38,34,0,3,0,3,3,0
"""LN""","""DJI_0788""",2023-02-13 06:49:44,42,38,34,0,27,0,27,19,8
"""LN""","""DJI_0789""",2023-02-13 06:49:50,42,38,34,0,41,0,41,41,0


In [14]:
df.filter((pl.col('Beach') == 'DC') & (pl.col('Seal Conf Lvl') == 14) & (pl.col('Clump Conf Lvl') == 58) & (pl.col('Overlap') == 42))

Beach,Image,Date,Seal Conf Lvl,Clump Conf Lvl,Overlap,Number of Clumps,Number of Seals,Number of Seals from Clumps,Combined,Ground Truth,Abs Diff
str,str,datetime[ns],i64,i64,i64,i64,i64,i64,i64,i64,i64
"""DC""","""DJI_0124""",2023-02-18 07:00:53,14,58,42,0,127,0,127,126,1
"""DC""","""DJI_0129""",2023-02-18 07:01:20,14,58,42,0,53,0,53,52,1
"""DC""","""DJI_0177""",2023-02-18 07:09:46,14,58,42,0,91,0,91,100,9
"""DC""","""DJI_0209""",2023-02-18 07:13:45,14,58,42,0,44,0,44,46,2
"""DC""","""DJI_0223""",2023-02-18 07:15:28,14,58,42,0,20,0,20,7,13


In [15]:
df.group_by(["Beach", "Seal Conf Lvl", "Clump Conf Lvl", "Overlap"]).agg([
          pl.max("Abs Diff").alias("Mean Abs Diff")
      ]).sort("Mean Abs Diff").group_by("Beach").agg([
               pl.first("Seal Conf Lvl").alias("Best Seal Conf Lvl"),
               pl.first("Clump Conf Lvl").alias("Best Clump Conf Lvl"),
               pl.first("Overlap").alias("Best Overlap"),
               pl.first("Mean Abs Diff").alias("Smallest Mean Abs Diff")
           ])

Beach,Best Seal Conf Lvl,Best Clump Conf Lvl,Best Overlap,Smallest Mean Abs Diff
str,i64,i64,i64,i64
"""DC""",18,42,46,12
"""LN""",42,74,18,4
"""LS""",10,46,90,104


In [16]:
df.filter((pl.col('Beach') == 'LN') & (pl.col('Seal Conf Lvl') == 42) & (pl.col('Clump Conf Lvl') == 74) & (pl.col('Overlap') == 18))

Beach,Image,Date,Seal Conf Lvl,Clump Conf Lvl,Overlap,Number of Clumps,Number of Seals,Number of Seals from Clumps,Combined,Ground Truth,Abs Diff
str,str,datetime[ns],i64,i64,i64,i64,i64,i64,i64,i64,i64
"""LN""","""DJI_0781""",2023-02-13 06:48:24,42,74,18,0,46,0,46,50,4
"""LN""","""DJI_0783""",2023-02-13 06:48:36,42,74,18,0,12,0,12,9,3
"""LN""","""DJI_0786""",2023-02-13 06:49:08,42,74,18,0,3,0,3,3,0
"""LN""","""DJI_0788""",2023-02-13 06:49:44,42,74,18,0,23,0,23,19,4
"""LN""","""DJI_0789""",2023-02-13 06:49:50,42,74,18,0,37,0,37,41,4


In [17]:
df.filter((pl.col('Beach') == 'DC') & (pl.col('Seal Conf Lvl') == 18) & (pl.col('Clump Conf Lvl') == 42) & (pl.col('Overlap') == 46))

Beach,Image,Date,Seal Conf Lvl,Clump Conf Lvl,Overlap,Number of Clumps,Number of Seals,Number of Seals from Clumps,Combined,Ground Truth,Abs Diff
str,str,datetime[ns],i64,i64,i64,i64,i64,i64,i64,i64,i64
"""DC""","""DJI_0124""",2023-02-18 07:00:53,18,42,46,0,124,0,124,126,2
"""DC""","""DJI_0129""",2023-02-18 07:01:20,18,42,46,0,52,0,52,52,0
"""DC""","""DJI_0177""",2023-02-18 07:09:46,18,42,46,0,91,0,91,100,9
"""DC""","""DJI_0209""",2023-02-18 07:13:45,18,42,46,0,34,0,34,46,12
"""DC""","""DJI_0223""",2023-02-18 07:15:28,18,42,46,0,19,0,19,7,12
